[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/koya-jp/AA-google-colab-kohya/blob/master/webui_sdxl_run_v1.6.0.ipynb)

#### *** 환경준비 (터미널에서 실행) ***

In [ ]:
# 신규 환경 만들기 
conda create -y --name sd_env python=3.10.6
conda activate sd_env
conda install -y ipykernel

# 환경 재설정 (삭제)
conda activate sd_env
conda deactivate
conda remove -y -n sd_env --all

#### *** 환경구축 ***

In [ ]:
%cd /home/studio-lab-user/
!python -V
# !df -h
# !rm -rf /home/studio-lab-user/.cache # 캐시삭제 

###############################################################################

%conda install -y aria2==1.36.0
%conda install -y -c conda-forge libglib==2.78.0
!pip install torchvision==0.15.2 -U
!pip install xformers==0.0.20 -U
!pip install triton==2.0.0 -U
!pip install httpx==0.24.1 -U
!pip install matplotlib==3.8.1 -U
!pip install ipython==8.17.2 -U
# !pip install torch==2.0.0+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U

import os
import glob
from IPython import get_ipython

# 그래프 그리기 라이브러리를 인라인으로 표시하는 설정 
get_ipython().run_line_magic('matplotlib', 'inline')

# Stable Diffusion WebUI 리포지토리 없으면 복제 
if not os.path.isdir("/home/studio-lab-user/ui"):
    %cd /home/studio-lab-user
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /home/studio-lab-user/ui
    # # Stable Diffusion WebUI v1.9.4 사용
    # %cd /home/studio-lab-user/ui
    # !git checkout master
    # !git checkout feee37d75f1b168768014e4634dcb156ee649c05

In [ ]:
# 업데이트 
%cd /home/studio-lab-user/ui
!git checkout master
!git pull #업데이트 

#### ***⭐ 실행 ----------------------------------------------------------------------------***

In [ ]:
%cd /home/studio-lab-user/ui
!rm -rf /home/studio-lab-user/.cache #⭐캐시삭제 
!df -h #현재 여유공간 표시 

# Model이 없으면 blue_pencil-XL-v0.8.0.safetensors을 다운로드함
if not glob.glob("/home/studio-lab-user/ui/models/Stable-diffusion/*.safetensors"):
    os.system("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.8.0.safetensors -d /home/studio-lab-user/ui/models/Stable-diffusion -o blue_pencil-XL-v0.8.0.safetensors")

# 설정변경 
%cd /home/studio-lab-user/ui
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /home/studio-lab-user/ui/modules/shared_options.py

# SDXL용 UI 편집
if os.path.exists("/home/studio-lab-user/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /home/studio-lab-user/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /home/studio-lab-user/ui/ui-config.json

if os.path.exists("/home/studio-lab-user/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /home/studio-lab-user/ui/config.json

# NGROK TOKEN 정의 
ngrok_token="YOUR NGROK TOKEN"

# Stable Diffusion WebUI 시작 
ARGS = "\"--listen --xformers --enable-insecure-extension-access --no-half-vae --ngrok " + ngrok_token + "\""
!COMMANDLINE_ARGS=$ARGS python launch.py

#### *** 이미지 생성 결과 Zip으로 저장 ***

In [ ]:
# 이미지 생성결과를 ZIP으로 생성하고 100개 이상의 폴더는 100개마다 ZIP으로 생성함 

import os
import glob
import datetime
import zipfile
import shutil

# 오늘날짜 받기 
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 압축할 이미지 경로 
# 이미지 경로 
img_paths = [f"/home/studio-lab-user/ui/outputs/{folder}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# ZIP파일경로 
zip_paths = [f"/home/studio-lab-user/outputs_{folder}_{today}.zip" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]

# 이동대상 폴더 경로 
move_paths = [f"{img_path}/*" for img_path in img_paths]

# 불필요한 폴더 경로 
delete_paths = [f"/home/studio-lab-user/Delete/{folder}/{today}" for folder in ["txt2img-images", "txt2img-grids", "img2img-images", "img2img-grids", "extras-images"]]


# ZIP파일로 압축하는 기능 
def zip_files(img_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        # 하위폴더와 파일의 모든 경로를 가져옴 
        for root, dirs, files in os.walk(img_path):
            for file in files:
                # ZIP파일에 쓸 경로를 상대경로 설정하기 
                rel_path = os.path.relpath(os.path.join(root, file), img_path)
                # ZIP파일로 쓰기 
                zf.write(os.path.join(root, file), rel_path)


# img_path의 파일수가 50개 이상이면 1,2와 같은 이름으로 폴더를 만들어 100개씩 이동시키는 함수
def split_files(img_path):
    # img_path 파일수 조회 
    file_count = len(os.listdir(img_path))

    # 파일수가 50개 이상이면 
    if file_count > 50:
        # 폴더번호 초기화 
        folder_num = 1
        # 파일목록 가져오기 
        file_list = os.listdir(img_path)
        # 파일목록 100개씩 분할 
        file_chunks = [file_list[i:i+50] for i in range(0, file_count, 50)]
        # 분할된 파일목록별로 처리 
        for file_chunk in file_chunks:
            # 폴더명 만들기 
            folder_name = str(folder_num)
            # 폴더 경로 생성 
            folder_path = os.path.join(img_path, "zip_"+folder_name)
            # 폴더가 없으면 생성 
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            # 폴더로 파일이동 
            for file in file_chunk:
                file_path = os.path.join(img_path, file)
                shutil.move(file_path, folder_path)
            # 폴더번호 업데이트 
            folder_num += 1


# 이미지 경로와 ZIP파일 경로의 쌍으로 압축 및 이동을 수행하는 함수
def zip_and_move(img_path, zip_path, move_path, delete_path):
    # 이미지 경로에 폴더나 파일이 없으면 건너뜀 

    try:
        if len(os.listdir(img_path)) == 0:
            return
    except FileNotFoundError:
        return

    # img_path의 파일수가 100개 이상이면 1,2와 같은 이름으로 폴더를 만들어 100개씩 이동시키는 함수를 호출함
    split_files(img_path)

    # img_path 하위폴더 목록 가져오기 
    subfolder_list = os.listdir(img_path)

    # 하위폴더별로 처리 
    for subfolder in subfolder_list:
        # 하위폴더 경로 생성 
        subfolder_path = os.path.join(img_path, subfolder)
        # ZIP파일명 생성 (하위폴더명끝에 추가)
        zip_name = zip_path[:-4] + "_" + subfolder + ".zip"
        # ZIP파일로 압축하는 함수호출 
        zip_files(subfolder_path, zip_name)

    # 이동할 폴더가 없으면 생성 
    if not os.path.exists(delete_path):
        os.makedirs(delete_path)

    # 와일드카드와 일치하는 경로목록 가져옴 
    move_list = glob.glob(move_path)

    # 이동 실행 
    for file in move_list:
        shutil.move(file, delete_path)

    print(img_path)
    !ls {img_path}


# 용령 정리 
!rm -rf /home/studio-lab-user/.cache # 캐시삭제 

# 이미지경로와 ZIP파일경로의 쌍으로 압축 및 이동을 수행
for img_path, zip_path, move_path, delete_path in zip(img_paths, zip_paths, move_paths, delete_paths):
    zip_and_move(img_path, zip_path, move_path, delete_path)

# 예비 
# today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# zip_path = f"/home/studio-lab-user/outputs-{today}.zip"
# img_path = "Delete/txt2img-images/20230816_130141/2023-08-16/2"
# zip_files("/home/studio-lab-user/"+img_path,zip_path)

#### ***용량정리***

In [ ]:
# 용량확인 
!df -h
!free -h
%cd ~
!ls -a .bashrc .config .ipython .jupyter .local .ngrok2 .nv .sagemaker .virtual_documents .wget-hsts

In [ ]:
# 용량정리 
!rm -rf /home/studio-lab-user/.ipynb_checkpoints
!rm -rf /home/studio-lab-user/.cache # 캐시삭제 
!rm -rf /home/studio-lab-user/.virtual_documents 
!rm -rf /home/studio-lab-user/.nv 
# !rm -rf /home/studio-lab-user/.conda #리셋(환경 다시 만들려면 실행) 
!ls -la /home/studio-lab-user/
!df -h

#### ***Model, VAE***

In [ ]:
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
!rm -rf /home/studio-lab-user/.cache #⭐캐시삭제 

# Model
model_dir="/home/studio-lab-user/ui/models/Stable-diffusion"
!rm -rf {model_dir}/* #⭐리셋 
# !rm -rf {model_dir}/beautifulRealistic_v60.safetensors #⭐削除したいファイルを入力

# SDXL - 리얼 
# !{lode_mode} https://civitai.com/api/download/models/157378 -d {model_dir} -o fudukiMix_v15.safetensors
!{lode_mode} https://civitai.com/api/download/models/161009 -d {model_dir} -o CherryPickerXL26.safetensors
# !{lode_mode} https://civitai.com/api/download/models/167881 -d {model_dir} -o AfterRealXL_beta2.safetensors
# !{lode_mode} https://civitai.com/api/download/models/145196 -d {model_dir} -o MiaModel_v3.safetensors
# !{lode_mode} https://civitai.com/api/download/models/163192 -d {model_dir} -o XXMix_9realisticSDXL_v1.71.safetensors

# SDXL - 애니메이션 
# !{lode_mode} https://civitai.com/api/download/models/155870 -d {model_dir} -o sdvn7Nijistylexl_v1.safetensors
# !{lode_mode} https://civitai.com/api/download/models/145893 -d {model_dir} -o BreakDomainXL.safetensors

# SD2.0 이전 
# !{lode_mode} https://civitai.com/api/download/models/119057 -d {model_dir} -o MeinaMix.safetensors
# !{lode_mode} https://civitai.com/api/download/models/113479 -d {model_dir} -o beautifulRealistic_v60.safetensors

# VAE
vae_dir="/home/studio-lab-user/ui/models/VAE"
!rm {vae_dir}/* #⭐리셋 
!{lode_mode} https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d {vae_dir} -o sdxl_vae.vae.safetensors
# !{lode_mode} https://huggingface.co/okozk/custom_models/resolve/main/vae_mse_840000_ema_semi.pt -d {vae_dir} -o vae_mse_840000_ema_semi.vae.pt

# 확인 
!echo -= model_dir 확인 =- && ls -l {model_dir}
!echo -= vae_dir 확인 =- && ls -l {vae_dir}
!echo -= 여유공간 확인 =- && df -h

#### ***Lora、embedings***

In [ ]:
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"
!rm -rf /home/studio-lab-user/.cache #⭐캐시삭제 
lora_dir="/home/studio-lab-user/ui/models/Lora" 
embeddings_dir="/home/studio-lab-user/ui/embeddings"

# lora ######################################################################
!rm -rf {lora_dir}/* #⭐리셋 
# !{lode_mode} https://civitai.com/api/download/models/134909 -d {lora_dir} -o japanese-girl-sdxl.safetensors
# !{lode_mode} https://civitai.com/api/download/models/121538 -d {lora_dir} -o watercolor-style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/136749 -d {lora_dir} -o ChibiStyleXL.safetensors
# !{lode_mode} https://civitai.com/api/download/models/138938 -d {lora_dir} -o Linear-Manga-Style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/137124 -d {lora_dir} -o DreamART-Style.safetensors
# !{lode_mode} https://civitai.com/api/download/models/126807 -d {lora_dir} -o Photorealistic-Slider.safetensors
# !{lode_mode} https://civitai.com/api/download/models/127510 -d {lora_dir} -o greg_rutkowski_xl_2.safetensors
# !{lode_mode} https://civitai.com/api/download/models/118427 -d {lora_dir} -o perfect-eyes-xl.safetensors
!{lode_mode} https://civitai.com/api/download/models/62833 -d {lora_dir} -o detail-tweaker-xl.safetensors

# embedings ######################################################################
# !rm -rf {embeddings_dir}/* #⭐리셋 
# !{lode_mode} https://huggingface.co/okozk/ComfyUI/resolve/main/unaestheticXLv13.safetensors -d {embeddings_dir} -o unaestheticXLv13.safetensors # 아름다움 
!{lode_mode} https://civitai.com/api/download/models/134424 -d {embeddings_dir} -o unaestheticX-NegL.safetensors # Flat2처럼 사용 
!{lode_mode} https://civitai.com/api/download/models/134583 -d {embeddings_dir} -o negativeXL_D.safetensors # 노출도 네거티브 0.5~0.7정도 
!{lode_mode} https://civitai.com/api/download/models/139645 -d {embeddings_dir} -o peopleneg.safetensors # 인간의 부정적인 특징 학습 실사용 
!{lode_mode} https://civitai.com/api/download/models/128882 -d {embeddings_dir} -o SimplepositiveXLv1.safetensors # Simple 긍정적인 요소 

# 확인 ######################################################################
!echo -= lora_dir 확인 =- && ls -l {lora_dir}
!echo -= embeddings_dir 확인 =- && ls -l {embeddings_dir}
!echo -= 여유공간 확인 =- && df -h

#### ***확장기능 설정***

In [ ]:
# 각 디렉토리 
base_dir="/home/studio-lab-user/ui"
extensions_dir = base_dir + "/extensions"
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"

# 확장기능 추가  ######################################################################
!rm -rf {extensions_dir}/* #⭐리셋 

#⭐삭제할 파일이력 
%cd {extensions_dir}
# !rm -rf sd_lama_cleaner controlnet wd14-tagger rembg depth-lib two-shot photopea-embed multidiffusion-upscaler model-toolkit haku-img cutoff Config-Presets  Styles-Editor

# 설치 
# !git clone https://github.com/camenduru/sd-civitai-browser {extensions_dir}/civitai-browser
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {extensions_dir}/images-browser
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing {extensions_dir}/infinite-image-browsing
!git clone https://github.com/Mikubill/sd-webui-controlnet {extensions_dir}/controlnet
!git clone https://github.com/IDEA-Research/DWPose {extensions_dir}/DWPose
!git clone https://github.com/anonCantCode/sd-webui-send-to-controlnet {extensions_dir}/send-to-controlnet
# !git clone https://github.com/jexom/sd-webui-depth-lib {extensions_dir}/depth-lib
!git clone https://github.com/richrobber2/canvas-zoom {extensions_dir}/canvas-zoom
!git clone https://github.com/huchenlei/sd-webui-openpose-editor {extensions_dir}/openpose-editor
# !git clone https://github.com/fkunn1326/openpose-editor {extensions_dir}/openpose-editor
# !git clone https://github.com/hnmr293/posex {extensions_dir}/posex
# !git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor {extensions_dir}/3d-open-pose-editor
# !git clone https://github.com/camenduru/tunnels {extensions_dir}/tunnels
# !git clone https://github.com/etherealxx/batchlinks-webui {extensions_dir}/batchlinks-webui
# !git clone https://github.com/catppuccin/stable-diffusion-webui {extensions_dir}/catppuccin
!git clone https://github.com/Haoming02/sd-webui-prompt-format {extensions_dir}/prompt-format
# !git clone https://github.com/eideehi/sd-webui-better-prompt {extensions_dir}/better-prompt
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {extensions_dir}/tagcomplete
# !git clone https://github.com/chrisgoringe/Styles-Editor {extensions_dir}/Styles-Editor
# !git clone https://github.com/opparco/stable-diffusion-webui-two-shot {extensions_dir}/two-shot
# !git clone https://github.com/bbc-mc/sdweb-clip-changer {extensions_dir}/clip-changer
# !git clone https://github.com/hnmr293/sd-webui-cutoff {extensions_dir}/cutoff
# !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger {extensions_dir}/wd14-tagger
!git clone https://github.com/Bing-su/adetailer {extensions_dir}/adetailer
# !git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 {extensions_dir}/multidiffusion-upscaler
!git clone https://github.com/alemelis/sd-webui-ar {extensions_dir}/sd-webui-ar
# !git clone https://github.com/Zyin055/Config-Presets {extensions_dir}/Config-Presets
# !git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-haku-img  {extensions_dir}/haku-img
# !git clone https://github.com/yankooliveira/sd-webui-photopea-embed  {extensions_dir}/photopea-embed
# !git clone https://github.com/arenasys/stable-diffusion-webui-model-toolkit {extensions_dir}/model-toolkit
# !git clone https://github.com/aka7774/sd_katanuki {extensions_dir}/sd_katanuki
# !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg {extensions_dir}/rembg
!git clone https://github.com/ljleb/sd-webui-freeu {extensions_dir}/freeu
# !git clone https://github.com/aka7774/sd_lama_cleaner {extensions_dir}/sd_lama_cleaner


# 리셋 
!git -C /home/studio-lab-user/ui/extensions reset --hard
!git -C /home/studio-lab-user/ui/repositories/stable-diffusion-stability-ai reset --hard

# %cd {extensions_dir}/prompt-all-in-one
# !git checkout a46e7f24a9aa4817a6fe81647aa2c85be096f72b

# sd-webui-ar의 확장기능에 SDXL권장크기 쓰기 
%cd {extensions_dir}
f = open("/home/studio-lab-user/ui/extensions/sd-webui-ar/resolutions.txt", "w")
text = """1, 1024, 1024 # 1:1 Square
2, 886, 1182 # 3:4 Photo
3, 836, 1254 # 2:3 Landscape
4, 768, 1344 # 9:16 Vertical"""
f.write(text)
f.close()

# wd14-tagger오류대책（커밋 → 99bf7d8132cb757edbcdfaad654f31f32f3500a5）
# !sed -i -e 's/from webui import wrap_gradio_gpu_call/from modules.call_queue import wrap_gradio_gpu_call/g' /home/studio-lab-user/ui/extensions/wd14-tagger/tagger/ui.py
# !sed -i -e 's/from modules.shared import models_path/import argparse\nimport os\n\nmodules_path = os.path.dirname(os.path.realpath(__file__))\n\nparser_pre = argparse.ArgumentParser(add_help=False)\nparser_pre.add_argument("--data-dir", type=str, default=os.path.dirname(modules_path), help="base path where all user data is stored", )\ncmd_opts_pre = parser_pre.parse_known_args()[0]\ndata_path = cmd_opts_pre.data_dir\n\nmodels_path = os.path.join(data_path, "models")/g' /home/studio-lab-user/ui/extensions/wd14-tagger/preload.py

# 확인 ######################################################################
!echo -= extensions_dir 확인 =- && ls -l {extensions_dir}
!echo -= 여유공간확인 =- && df -h

#### ***ControlNet Models 설치 ***

In [ ]:
# 각 디렉토리 
base_dir="/home/studio-lab-user/ui"
ctn_model_dir = base_dir + "/models/ControlNet"
lode_mode="aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"

# SDXL지원버전 ControlNet모델추가  ######################################################################
!rm -rf {ctn_model_dir}/* #⭐리셋 
#⭐삭제할 파일입력
%cd {ctn_model_dir}
# !rm -rf ip-adapter_xl.pth t2i-adapter_diffusers_xl_lineart.safetensors control-lora-canny-rank256.safetensors ip-adapter_xl.pth
# 설치 
!{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d {ctn_model_dir} -o control-lora-canny-rank256.safetensors
# !{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d {ctn_model_dir} -o control-lora-depth-rank256.safetensors
# !{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d {ctn_model_dir} -o control-lora-recolor-rank256.safetensors
# !{lode_mode} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d {ctn_model_dir} -o control-lora-sketch-rank256.safetensors
!{lode_mode} https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors -d {ctn_model_dir} -o thibaud_xl_openpose_256lora.safetensors
# !{lode_mode} https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth -d {ctn_model_dir} -o ip-adapter_xl.pth
# !{lode_mode} https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors -d {ctn_model_dir} -o sargezt_xl_softedge.safetensors #2.5GB

# 확인  ######################################################################
!echo -= ctn_model_dir 확인 =- && ls -l {ctn_model_dir}
!echo -= 여유공간 확인 =- && df -h